In [ ]:
import pickle
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
from shapely.ops import MultiLineString, LineString
import geopandas as gpd

from bikewaysim.paths import config
from bikewaysim.impedance_calibration import speedfactor, stochastic_optimization
from bikewaysim.map_matching import map_match, post_process
from bikewaysim.network import prepare_network, modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs

In [ ]:
# with (config['calibration_fp']/'ready_for_calibration.pkl').open('rb') as fh:
#     match_results = pickle.load(fh)
with (config['matching_fp'] / f'match_dict_full_5.pkl').open('rb') as fh:
    match_dict = pickle.load(fh)

cutoff = 0.90 # set pct of points that need to be matched
above_threshold, below_threshold, failed_matches, match_ratios = post_process.mapmatch_results(match_dict,cutoff)
match_dict = {key:item for key,item in match_dict.items() if key in above_threshold}

In [ ]:
#try doing this in rustworkx and time the difference
import rustworkx as rx

links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)
base_impedance_col = "travel_time_min"

In [ ]:
rustworkx_routing_funcs.back_to_base_impedance(base_impedance_col,links,turns,turn_G)
links.set_index(['linkid','reverse_link'],inplace=True,drop=False)
match_results = {}
#shortest_results = {}
failed_shortest_path = []

In [ ]:
# get list of ods from match_dict?
# match_results
keys = list(match_dict.keys())[40]

In [ ]:
from importlib import reload
reload(rustworkx_routing_funcs)
starts, ends = post_process.get_ods_from_match_dict(match_dict,links)

In [ ]:
added_nodes = rustworkx_routing_funcs.add_virtual_edges(starts,ends,links,turns,turn_G)

In [ ]:
shortest_paths = rustworkx_routing_funcs.rx_shortest_paths(list(zip(starts,ends)),turn_G)

In [ ]:
# first get the connecting links
starting_edges = turns.loc[turns['source_A'].isin(set(starts)),['source_A','source_linkid','source_reverse_link']].drop_duplicates().values
ending_edges = turns.loc[turns['target_B'].isin(set(ends)),['target_linkid','target_reverse_link','target_B']].drop_duplicates().values

# get tuples of the edges we need to add
starting_edges = [(int(x[0]),(int(x[1]),bool(x[2])),link_costs.get((x[1],x[2]))) for x in starting_edges]
ending_edges = [((int(x[0]),bool(x[1])),int(x[2]),0) for x in ending_edges]

# this makes sure we don't add any duplicate nodes to the graph
new_nodes = list(set(starts + ends) - set(turn_rx.nodes()))
turn_rx.add_nodes_from(new_nodes)

# create a dict for mapping back and forth (only valid if we're positive that each node value is unique)
node_to_idx = dict(zip(turn_rx.nodes(),turn_rx.node_indices()))
idx_to_node = dict(zip(turn_rx.node_indices(),turn_rx.nodes()))

# get idx edges
starting_virtual_edges = [(node_to_idx[a],node_to_idx[b],{'weight':weight}) for a, b, weight in starting_edges] 
ending_virtual_edges = [(node_to_idx[a],node_to_idx[b],{'weight':weight}) for a, b, weight in ending_edges] 

# add these to graph
turn_rx.add_edges_from(starting_virtual_edges + ending_virtual_edges)


ods = [(node_to_idx[start],node_to_idx[end]) for start, end in zip(starts,ends)]
results_rx = [rx.dijkstra_shortest_paths(turn_rx,start,end,weight_fn=lambda x: x['weight']) for start, end in ods]
converted = [[idx_to_node[x] for x in i[1:-1]] for sublist in results_rx for i in sublist.values()]

In [ ]:
x = list(results_rx[0].values())
turn_rx.get_all_edge_data(x[0],x[1])

In [ ]:
betas = [-0.5]
betas_tup = (
    {'col':'bike lane','type':'link','range':[-1,0]},
)

import numpy as np
import networkx as nx

#update link costs (in the 'link_cost' column)
stochastic_optimization.link_impedance_function(betas, betas_tup, links, base_impedance_col)


In [ ]:
links['link_cost_override'] = False

# override the cost with 9e9 if feature is a future off-street facility
# this effectively prevents routing w/o messing around with the network structure
links.loc[links['link_cost_override']==True,'link_cost'] = 9e9

#create cost dict (i think this is the fastest python way to do this?)
tuple_index = tuple(zip(links['linkid'],links['reverse_link']))
cost_dict = dict(zip(tuple_index,links['link_cost']))

#costs are stored in the turn graph (only target matters, initial link cost is added during routing)
turns['target_link_cost'] = turns[['target_linkid','target_reverse_link']].apply(lambda x: cost_dict.get(tuple(x.values),False),axis=1)

#update turn costs
stochastic_optimization.turn_impedance_function(betas, betas_tup, turns)

#cacluate new total cost
turns['total_cost'] = (turns['target_link_cost'] + turns['turn_cost'])


In [ ]:
if turns['total_cost'].isna().any():
    raise Exception("There are nan edge costs, exiting")

#check for negative link impedance
if (links['link_cost'] < 0).any() | (turns['total_cost'] < 0).any():
    return False

#update turn network graph with final cost
cols = ['source_linkid','source_reverse_link','target_linkid','target_reverse_link','total_cost']
# updated_edge_costs = {((row[0],row[1]),(row[2],row[3])):row[4] for row in turns[cols].itertuples(index=False)}

updated_edge_costs = [((row[0],row[1]),(row[2],row[3]),row[4]) for row in turns[cols].values]

updated_edge_costs = [(node_to_idx[x[0]],node_to_idx[x[1]],{'weight':x[2]}) for x in updated_edge_costs] 

# updates the edges
_ = [turn_rx.update_edge(*x) for x in updated_edge_costs]


In [ ]:
turn_rx.edges()[0]

In [ ]:
updated_edge_costs

In [ ]:
turn_rx.get_edge_data(0,1)

In [ ]:
# want to assign edges new weights
turn_rx.edges()[0]

In [ ]:
turn_rx[turn_rx.edge_indices_from_endpoints(0,1)]

In [ ]:
converted[0]

In [ ]:
`#get length
results_nx[0]

In [ ]:
test = [other_way.get(x) for x in list(first_path)]
test

In [ ]:
list(results_nx.keys())

In [ ]:
import networkx as nx
results_nx = []
for start, end in zip(start_nodes,end_nodes):
    start = int(start)
    end = int(end)
    results_nx.append(nx.single_source_dijkstra(turn_G,start,end,weight='weigth'))

In [ ]:
results_nx = {}
for tripid, items in tqdm(match_dict.items()):

    #get start and end linkid
    start = tuple(match_dict[tripid]['edges'].iloc[0,:].values)
    end = tuple(match_dict[tripid]['edges'].iloc[-1,:].values)

    #get start and end node for shortest and impedance routing
    # #TODO change this to be live so we don't run into errors when the matching network is different
    # start = links.loc[start,'A']
    # end = links.loc[end,'B']

    results_nx[tripid] = 